In [17]:
from openai import OpenAI
openai_client = OpenAI()


In [18]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [19]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)


In [20]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
    )

    return results


In [21]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [ ]:
instructions = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

If you want to look up the answer, explain why before making the call
""".strip()

tools = [search_tool]

question = 'I just discovered the course. Can I still join it?'

In [29]:
import json

def make_call(call):
    f_name = call.name
    arguments = json.loads(call.arguments)

    if f_name == 'search':
        results = search(**arguments)
    # if you add another function, we can put it here
    # elif f_name == 'add_entry':
    #    results = add_entry(**arguments)
    else:
        raise ValueError(f'unknown function {f_name}')

    json_results = json.dumps(results)

    return {
        "type": "function_call_output",
        "call_id": call.call_id,
        "output": json_results,
    }

In [32]:
question = 'I just discovered the course. Can I still join it?'

chat_messages = [
    {"role": "developer", "content": instructions},
    {"role": "user", "content": question}
]

while True:
    response = openai_client.responses.create(
        model='gpt-4o-mini',
        input=chat_messages,
        tools=tools
    )

    has_function_calls = False

    # Add response to chat history for LLM's "memory"
    chat_messages.extend(response.output)

    for entry in response.output:
        if entry.type == "function_call":
            print('Function call:')
            print(entry)
            result = make_call(entry)
            print('   ', 'Output:')
            print('   ', result['output'])
            chat_messages.append(result)
            has_function_calls = True
            print()

        elif entry.type == "message":
            print('Assistant:')
            print(entry.content[0].text)
            print()

    if not has_function_calls:
        break 


Assistant:
To answer your question accurately, I need to check the course FAQ for information about enrollment deadlines and whether late registration is allowed. Let me look that up for you.

Function call:
ResponseFunctionToolCall(arguments='{"query":"join course late enrollment registration"}', call_id='call_PnHrySJ5pfo642Qg2jzk5SCB', name='search', type='function_call', id='fc_0fda7a92be71ccb50068efc225fa088194aa42aecd443a4ed9', status='completed')
    Output:
    [{"text": "No, late submissions are not allowed. But if the form is still not closed and it\u2019s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y\nOlder news:[source1] [source2]", "section": "General course-related questions", "question": "Homework - Are late submissions of homework allowed?", "course": "data-engineering-zoomcamp"}, {"text": "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that th